# Aggregating DataFrames in PySpark HW

First let's start up our PySpark instance

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HW').getOrCreate()

In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Read in the dataFrame for this Notebook

In [3]:
airbnb = spark.read.csv('Datasets/nyc_air_bnb.csv',inferSchema=True,header=True)

## About this dataset

This dataset describes the listing activity and metrics for Air BNB bookers in NYC, NY for 2019. Each line in the dataset is a booking. 

**Source:** https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/data

Let's go ahead and view the first few records of the dataset so we know what we are working with.

In [4]:
airbnb.limit(5).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Now print the schema so we can make sure all the variables have the correct types

In [5]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



Notice here that some of the columns that are obviously numeric have been incorrectly identified as "strings". Let's edit that. Otherwise we cannot aggregate any of the numeric columns.

In [9]:
airbnb = airbnb.withColumn('price', airbnb.price.cast(IntegerType())) \
    .withColumn('minimum_nights', airbnb.minimum_nights.cast(IntegerType())) \
    .withColumn('number_of_reviews', airbnb.number_of_reviews.cast(IntegerType())) \
    .withColumn('reviews_per_month', airbnb.reviews_per_month.cast(IntegerType())) \
    .withColumn('calculated_host_listings_count', airbnb.calculated_host_listings_count.cast(IntegerType())) \
    .withColumn('latitude', airbnb.latitude.cast(DoubleType())) \
    .withColumn('longitude', airbnb.longitude.cast(DoubleType()))

### Alright now we are ready to dig in!


### 1. How many rows are in this dataset?

In [10]:
airbnb.count()

49079

### 2. How many total reviews does each host have?

In [11]:
airbnb.groupBy('host_id').sum('number_of_reviews').show(5)

+-------+----------------------+
|host_id|sum(number_of_reviews)|
+-------+----------------------+
| 716306|                   197|
|1203500|                    35|
| 368528|                     1|
|1577493|                    16|
|1390555|                    50|
+-------+----------------------+
only showing top 5 rows



### 3. Show the min and max of all the numeric variables in the dataset

In [14]:
airbnb.select(
    'price',
    'minimum_nights',
    'number_of_reviews',
    'reviews_per_month',
    'calculated_host_listings_count',
    'latitude',
    'longitude').summary('min','max').show()

+-------+-----+--------------+-----------------+-----------------+------------------------------+---------+-----------+
|summary|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count| latitude|  longitude|
+-------+-----+--------------+-----------------+-----------------+------------------------------+---------+-----------+
|    min|  -74|             0|                0|                0|                             0|-74.16254|  -74.24442|
|    max|10000|          1250|              629|               58|                           365| 40.91306|2.4906403E7|
+-------+-----+--------------+-----------------+-----------------+------------------------------+---------+-----------+



### 4. Which host had the highest number of reviews?

Only display the top result.

Bonus: format the column names

In [16]:
airbnb.groupBy('host_id').agg(sum('number_of_reviews').alias('rev')).orderBy(desc('rev')).show(1)

+--------+----+
| host_id| rev|
+--------+----+
|37312959|2273|
+--------+----+
only showing top 1 row



### 5. On average, how many nights did most hosts specify for a minimum?

In [19]:
airbnb.agg(avg('minimum_nights').alias('min_night')).show()

+------------------+
|         min_night|
+------------------+
|7.1286126280910596|
+------------------+



### 6. What is the most expensive neighborhood to stay in on average?

Note: only show the one result

In [24]:
airbnb.groupBy('neighbourhood').agg(avg('price').alias('avg_price')).orderBy(desc('avg_price')).show(1)

+--------------+---------+
| neighbourhood|avg_price|
+--------------+---------+
|Fort Wadsworth|    800.0|
+--------------+---------+
only showing top 1 row



### 7. Display a two by two table that shows the average prices by room type (private and shared only) and neighborhood group (Manhattan and Brooklyn only)

In [30]:
airbnb.filter("room_type IN ('Private room','Shared room')").groupBy('room_type') \
    .pivot('neighbourhood_group', ['Manhattan','Brooklyn']).avg('price').show()

+------------+------------------+-----------------+
|   room_type|         Manhattan|         Brooklyn|
+------------+------------------+-----------------+
| Shared room| 89.06903765690376|50.52784503631961|
|Private room|116.05400302114803|76.47234042553191|
+------------+------------------+-----------------+



### Alright that's all folks!

### Great job!